# Example: BFV Basics

In [1]:
import seal
import example_helper

In this example, we demonstrate performing simple computations (a polynomial
evaluation) on encrypted integers using the BFV encryption scheme.

The first task is to set up an instance of the EncryptionParameters class.
It is critical to understand how the different parameters behave, how they
affect the encryption scheme, performance, and the security level. There are
three encryption parameters that are necessary to set:
  - `poly_modulus_degree`: degree of polynomial modulus
  - `coeff_modulus`: [ciphertext] coefficient modulus
  - `plain_modulus`: plaintext modulus; only for the BFV scheme 

The BFV scheme cannot perform arbitrary computations on encrypted data.
Instead, each ciphertext has a specific quantity called the `invariant noise
budget` -- or `noise budget` for short -- measured in bits. The noise budget
in a freshly encrypted ciphertext (initial noise budget) is determined by
the encryption parameters. Homomorphic operations consume the noise budget
at a rate also determined by the encryption parameters. In BFV the two basic
operations allowed on encrypted data are additions and multiplications, of
which additions can generally be thought of as being nearly free in terms of
noise budget consumption compared to multiplications. Since noise budget
consumption compounds in sequential multiplications, the most significant
factor in choosing appropriate encryption parameters is the multiplicative
depth of the arithmetic circuit that the user wants to evaluate on encrypted
data. Once the noise budget of a ciphertext reaches zero it becomes too
corrupted to be decrypted. Thus, it is essential to choose the parameters to
be large enough to support the desired computation; otherwise the result is
impossible to make sense of even with the secret key.

In [2]:
parms = seal.EncryptionParameters(seal.scheme_type.bfv)

The first parameter we set is the degree of the `polynomial modulus`. This
must be a positive power of 2, representing the degree of a power-of-two
cyclotomic polynomial; it is not necessary to understand what this means.

Larger `poly_modulus_degree` makes ciphertext sizes larger and all operations
slower, but enables more complicated encrypted computations. Recommended
values are 1024, 2048, 4096, 8192, 16384, 32768, but it is also possible
to go beyond this range.

In this example we use a relatively small polynomial modulus. Anything
smaller than this will enable only very restricted encrypted computations.

In [3]:
poly_modulus_degree = 4096
parms.poly_modulus_degree = poly_modulus_degree

Next we set the [ciphertext] `coefficient modulus` (coeff_modulus). This
parameter is a large integer, which is a product of distinct prime numbers,
each up to 60 bits in size. It is represented as a vector of these prime
numbers, each represented by an instance of the `Modulus` class. The
bit-length of coeff_modulus means the sum of the bit-lengths of its prime
factors.

A larger coeff_modulus implies a larger noise budget, hence more encrypted
computation capabilities. However, an upper bound for the total bit-length
of the coeff_modulus is determined by the `poly_modulus_degree`, as follows:

    | poly_modulus_degree | max coeff_modulus bit-length |
    |---------------------|------------------------------|
    | 1024                | 27                           |
    | 2048                | 54                           |
    | 4096                | 109                          |
    | 8192                | 218                          |
    | 16384               | 438                          |
    | 32768               | 881                          |

These numbers can also be found in `native/src/seal/util/hestdparms.h` encoded
in the function `SEAL_HE_STD_PARMS_128_TC`, and can also be obtained from the
function `CoeffModulus.MaxBitCount(poly_modulus_degree)`.

For example, if `poly_modulus_degree` is 4096, the `coeff_modulus` could consist
of three 36-bit primes (108 bits).

Microsoft SEAL comes with helper functions for selecting the coeff_modulus.
For new users the easiest way is to simply use `CoeffModulus.BFVDefault(poly_modulus_degree)`,
which returns `List[Modulus]` consisting of a generally good choice
for the given `poly_modulus_degree`.

In [4]:
parms.coeff_modulus = seal.CoeffModulus.BFVDefault(poly_modulus_degree)

The plaintext modulus can be any positive integer, even though here we take
it to be a power of two. In fact, in many cases one might instead want it
to be a prime number; we will see this in later examples. The plaintext
modulus determines the size of the plaintext data type and the consumption
of noise budget in multiplications. Thus, it is essential to try to keep the
plaintext data type as small as possible for best performance. The noise
budget in a freshly encrypted ciphertext is ~ `log2(coeff_modulus/plain_modulus)` (bits)
and the noise budget consumption in a homomorphic multiplication is of the
form `log2(plain_modulus) + (other terms)`.

The plaintext modulus is specific to the BFV scheme, and cannot be set when
using the CKKS scheme.

In [5]:
parms.plain_modulus = seal.Modulus(1024)

Now that all parameters are set, we are ready to construct a SEALContext
object. This is a heavy class that checks the validity and properties of the
parameters we just set.

In [6]:
context = seal.SEALContext(parms)
example_helper.print_parameters(context)

/
| Encryption parameters
| scheme: bfv
| poly_modulus_degree: 4096
| coeff_modulus size: 109(36 + 36 + 37) bits
| plain_modulus: 1024
\


The encryption schemes in Microsoft SEAL are public key encryption schemes.
For users unfamiliar with this terminology, a public key encryption scheme
has a separate public key for encrypting data, and a separate secret key for
decrypting data. This way multiple parties can encrypt data using the same
shared public key, but only the proper recipient of the data can decrypt it
with the secret key.

We are now ready to generate the secret and public keys. For this purpose
we need an instance of the KeyGenerator class. Constructing a KeyGenerator
automatically generates a secret key. We can then create as many public
keys for it as we want using `KeyGenerator::create_public_key`.

Note that `KeyGenerator::create_public_key` has another overload that takes
no parameters and returns a `Serializable<PublicKey>` object. We will discuss
this in `7_serialization.ipynb`.

In [7]:
keygen = seal.KeyGenerator(context)
secret_key = keygen.secret_key
public_key = keygen.create_public_key()

To be able to encrypt we need to construct an instance of Encryptor. Note
that the Encryptor only requires the public key, as expected. It is also
possible to use Microsoft SEAL in secret-key mode by providing the Encryptor
the secret key instead. We will discuss this in `7_serialization.ipynb`.

In [8]:
encryptor = seal.Encryptor(context, public_key)

Computations on the ciphertexts are performed with the Evaluator class. In
a real use-case the Evaluator would not be constructed by the same party
that holds the secret key.

In [9]:
evaluator = seal.Evaluator(context)

We will of course want to decrypt our results to verify that everything worked,
so we need to also construct an instance of Decryptor. Note that the Decryptor
requires the secret key.

In [10]:
decryptor = seal.Decryptor(context, secret_key)

As an example, we evaluate the degree 4 polynomial 

    4x^4 + 8x^3 + 8x^2 + 8x + 4

over an encrypted x = 6. The coefficients of the polynomial can be considered
as plaintext inputs, as we will see below. The computation is done modulo the
`plain_modulus` 1024.

While this examples is simple and easy to understand, it does not have much
practical value. In later examples we will demonstrate how to compute more
efficiently on encrypted integers and real or complex numbers.

Plaintexts in the BFV scheme are polynomials of degree less than the degree
of the polynomial modulus, and coefficients integers modulo the plaintext
modulus. For readers with background in ring theory, the plaintext space is
the polynomial quotient ring 

    Z_T[X]/(X^N + 1) 
    
where `N` is `poly_modulus_degree` and `T` is `plain_modulus`.

To get started, we create a plaintext containing the constant 6. For the
plaintext element we use a constructor that takes the desired polynomial as
a string with coefficients represented as hexadecimal numbers.

In [11]:
x = 6
x_plain = seal.Plaintext(example_helper.int_to_hex_string(x))
print(f"Express x = {x} as a plaintext polynomial 0x{x_plain.to_string()}.")

Express x = 6 as a plaintext polynomial 0x6.


We then encrypt the plaintext, producing a ciphertext. We note that the
`Encryptor::encrypt` function has another overload that takes as input only
a plaintext and returns a `Serializable<Ciphertext>` object. We will discuss
this in `7_serialization.ipynb`.

In [12]:
print("Encrypt x_plain to x_encrypted.")
x_encrypted = encryptor.encrypt(x_plain)

Encrypt x_plain to x_encrypted.


In Microsoft SEAL, a valid ciphertext consists of two or more polynomials
whose coefficients are integers modulo the product of the primes in the
coeff_modulus. The number of polynomials in a ciphertext is called its `size`
and is given by `Ciphertext::size()`. A freshly encrypted ciphertext always
has size 2. Also, there is plenty of noise budget left in this freshly encrypted ciphertext.

In [13]:
print(f"size of freshly encrypted x: {x_encrypted.size}")
print(f"noise budget in freshly encrypted x: {decryptor.invariant_noise_budget(x_encrypted)} bits")

size of freshly encrypted x: 2
noise budget in freshly encrypted x: 55 bits


We decrypt the ciphertext and print the resulting plaintext in order to
demonstrate correctness of the encryption.

In [14]:
x_decrypted = decryptor.decrypt(x_encrypted)
print(f"decryption of x_encrypted: 0x{x_decrypted.to_string()}")

decryption of x_encrypted: 0x6


When using Microsoft SEAL, it is typically advantageous to compute in a way
that minimizes the longest chain of sequential multiplications. In other
words, encrypted computations are best evaluated in a way that minimizes
the multiplicative depth of the computation, because the total noise budget
consumption is proportional to the multiplicative depth. For example, for
our example computation it is advantageous to factorize the polynomial as

    4x^4 + 8x^3 + 8x^2 + 8x + 4 = 4(x + 1)^2 * (x^2 + 1)

to obtain a simple depth 2 representation. Thus, we compute `(x + 1)^2` and
`(x^2 + 1)` separately, before multiplying them, and multiplying by 4.

First, we compute x^2 and add a plaintext "1". We can clearly see from the
print-out that multiplication has consumed a lot of noise budget. The user
can vary the plain_modulus parameter to see its effect on the rate of noise
budget consumption.

In [15]:
print("Compute x_sq_plus_one (x^2+1).")
x_sq_plus_one = evaluator.square(x_encrypted)
plain_one = seal.Plaintext("1")
evaluator.add_plain_inplace(x_sq_plus_one, plain_one)

Compute x_sq_plus_one (x^2+1).


Encrypted multiplication results in the output ciphertext growing in size.
More precisely, if the input ciphertexts have size M and N, then the output
ciphertext after homomorphic multiplication will have size M+N-1. In this
case we perform a squaring, and observe both size growth and noise budget
consumption.

In [16]:
print(f"size of x_sq_plus_one: {x_sq_plus_one.size}")
print(f"noise budget in x_sq_plus_one: {decryptor.invariant_noise_budget(x_sq_plus_one)} bits")

size of x_sq_plus_one: 3
noise budget in x_sq_plus_one: 33 bits


Even though the size has grown, decryption works as usual as long as noise
budget has not reached 0.

In [17]:
decrypted_result = decryptor.decrypt(x_sq_plus_one)
print(f"decryption of x_sq_plus_one: 0x{decrypted_result.to_string()}")

decryption of x_sq_plus_one: 0x25


In [18]:
print("Compute x_plus_one_sq ((x+1)^2).")

x_plus_one_sq = evaluator.add_plain(x_encrypted, plain_one)
evaluator.square_inplace(x_plus_one_sq)
print(f"size of x_plus_one_sq: {x_plus_one_sq.size}")
print(f"noise budget in x_plus_one_sq: {decryptor.invariant_noise_budget(x_plus_one_sq)} bits")

decrypted_result = decryptor.decrypt(x_plus_one_sq)
print(f"decryption of x_plus_one_sq: 0x{decrypted_result.to_string()}")

Compute x_plus_one_sq ((x+1)^2).
size of x_plus_one_sq: 3
noise budget in x_plus_one_sq: 33 bits
decryption of x_plus_one_sq: 0x31


In [19]:
plain_four = seal.Plaintext("4")
evaluator.multiply_plain_inplace(x_sq_plus_one, plain_four)
encrypted_result = evaluator.multiply(x_sq_plus_one, x_plus_one_sq)
print(f"size of encrypted_result: {encrypted_result.size}")
print(f"noise budget in encrypted_result: {decryptor.invariant_noise_budget(encrypted_result)} bits")
print("NOTE: Decryption can be incorrect if noise budget is zero.")

decrypted_result = decryptor.decrypt(encrypted_result)
print(f"decryption of encrypted_result: 0x{decrypted_result.to_string()}")

size of encrypted_result: 5
noise budget in encrypted_result: 4 bits
NOTE: Decryption can be incorrect if noise budget is zero.
decryption of encrypted_result: 0x54


Noise budget has reached 0, which means that decryption cannot be expected
to give the correct result. This is because both ciphertexts x_sq_plus_one
and x_plus_one_sq consist of 3 polynomials due to the previous squaring
operations, and homomorphic operations on large ciphertexts consume much more
noise budget than computations on small ciphertexts. Computing on smaller
ciphertexts is also computationally significantly cheaper.

`Relinearization` is an operation that reduces the size of a ciphertext after
multiplication back to the initial size, 2. Thus, relinearizing one or both
input ciphertexts before the next multiplication can have a huge positive
impact on both noise growth and performance, even though relinearization has
a significant computational cost itself. It is only possible to relinearize
size 3 ciphertexts down to size 2, so often the user would want to relinearize
after each multiplication to keep the ciphertext sizes at 2.

Relinearization requires special `relinearization keys`, which can be thought
of as a kind of public key. Relinearization keys can easily be created with
the KeyGenerator.

Relinearization is used similarly in both the BFV and the CKKS schemes, but
in this example we continue using BFV. We repeat our computation from before,
but this time relinearize after every multiplication.

In [20]:
relin_keys = keygen.create_relin_keys()

In [21]:
print("Compute and relinearize x_squared (x^2), then compute x_sq_plus_one (x^2+1).")

x_squared = evaluator.square(x_encrypted)
print(f"size of x_squared: {x_squared.size}")
evaluator.relinearize_inplace(x_squared, relin_keys)
print(f"size of x_squared (after relinearization): {x_squared.size}")
x_sq_plus_one = evaluator.add_plain(x_squared, plain_one)
print(f"noise budget in x_sq_plus_one: {decryptor.invariant_noise_budget(x_sq_plus_one)} bits")
decrypted_result = decryptor.decrypt(x_sq_plus_one)
print(f"decryption of x_sq_plus_one: 0x{decrypted_result.to_string()}")


Compute and relinearize x_squared (x^2), then compute x_sq_plus_one (x^2+1).
size of x_squared: 3
size of x_squared (after relinearization): 2
noise budget in x_sq_plus_one: 33 bits
decryption of x_sq_plus_one: 0x25


In [22]:
print("Compute x_plus_one (x+1), then compute and relinearize x_plus_one_sq ((x+1)^2).")

x_plus_one = evaluator.add_plain(x_encrypted, plain_one)
x_plus_one_sq = evaluator.square(x_plus_one)
print(f"size of x_plus_one_sq: {x_plus_one_sq.size}")
evaluator.relinearize_inplace(x_plus_one_sq, relin_keys)
print(f"size of x_plus_one_sq (after relinearization): {x_plus_one_sq.size}")
print(f"noise budget in x_sq_plus_one: {decryptor.invariant_noise_budget(x_plus_one_sq)} bits")
decrypted_result = decryptor.decrypt(x_plus_one_sq)
print(f"decryption of x_plus_one_sq: 0x{decrypted_result.to_string()}")

Compute x_plus_one (x+1), then compute and relinearize x_plus_one_sq ((x+1)^2).
size of x_plus_one_sq: 3
size of x_plus_one_sq (after relinearization): 2
noise budget in x_sq_plus_one: 33 bits
decryption of x_plus_one_sq: 0x31


In [23]:
print("Compute and relinearize encrypted_result (4(x^2+1)(x+1)^2).")
evaluator.multiply_plain_inplace(x_sq_plus_one, plain_four)
encrypted_result = evaluator.multiply(x_sq_plus_one, x_plus_one_sq)
print(f"size of encrypted_result: {encrypted_result.size}")
evaluator.relinearize_inplace(encrypted_result, relin_keys)
print(f"size of encrypted_result (after relinearization): {encrypted_result.size}")
print(f"noise budget in x_sq_plus_one: {decryptor.invariant_noise_budget(encrypted_result)} bits")
print("NOTE: Notice the increase in remaining noise budget.")
decrypted_result = decryptor.decrypt(encrypted_result)
print(f"decryption of encrypted_result: 0x{decrypted_result.to_string()}")

Compute and relinearize encrypted_result (4(x^2+1)(x+1)^2).
size of encrypted_result: 3
size of encrypted_result (after relinearization): 2
noise budget in x_sq_plus_one: 10 bits
NOTE: Notice the increase in remaining noise budget.
decryption of encrypted_result: 0x54
